# Pre-processing and Importing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/karthiksairam01/SemEval-Task9.git
!git clone https://github.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io.git

fatal: destination path 'SemEval-Task9' already exists and is not an empty directory.
fatal: destination path 'food-hazard-detection-semeval-2025.github.io' already exists and is not an empty directory.


In [ ]:
with open('/content/drive/My Drive/conceptnet-lite.csv', 'r') as f:
  file_contents = f.read()

In [ ]:
# Install necessary libraries
!pip install networkx pandas tqdm

!pip install -U pip setuptools wheel

In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
from tqdm import tqdm
import json

# Functions

In [ ]:
def get_titles_dict(incidents: pd.DataFrame, column: str) -> dict:
    try:
        values = np.unique(np.concatenate(incidents[column].values))
    except ValueError:
        values = np.unique(incidents[column].values)

    counts = np.array([sum([v in label for label in incidents[column].values]) for v in values])
    result = {}

    for v, n in zip(values, counts):
        doc = nlp(v)
        words = []
        for token in doc:
            if token.pos_ in {"NOUN","ADJ"}:
                words.append(token.text)
        result[v] = {"words": words}

    return result

def get_nodes_dict(incidents: pd.DataFrame, column: str) -> dict:
    try:
        values = np.unique(np.concatenate(incidents[column].values))
    except ValueError:
        values = np.unique(incidents[column].values)

    counts = np.array([sum([v in label for label in incidents[column].values]) for v in values])
    result = {}

    for v, n in zip(values, counts):
        doc = nlp(v)
        words = []
        for token in doc:
            if token.pos_ in {"NOUN", "PROPN", "VERB", "ADJ"}:
                words.append(token.text)
        result[v] = {"words": words}

    return result

def print_nodes(incidents: pd.DataFrame, column: str):
    try:
        values = np.unique(np.concatenate(incidents[column].values))
    except ValueError:
        values = np.unique(incidents[column].values)
    counts = np.array([sum([v in label for label in incidents[column].values]) for v in values])

    for v, n in zip(values, counts):
        # print(v)
        doc = nlp(v)
        words = []
        for token in doc:
            if token.pos_ in {"NOUN", "PROPN", "VERB", "ADJ"}:
                words.append(token.text)
        print(f'  {v}: {words}')

# Functions (A)

In [ ]:
def canonicalize_node(uri):
    """
    Canonicalizes a ConceptNet URI by removing parts after the lemma.
    For example:
    "/c/en/condiment/n" -> "/c/en/condiment"
    "/c/en/condiments/n" -> "/c/en/condiments"
    "/c/en/apple" stays "/c/en/apple"
    """
    parts = uri.strip('/').split('/')
    # We expect something like ["c", "en", "condiment", "n"]
    # Keep only the first three segments: c, language code, and the lemma
    if len(parts) >= 3:
        return "/" + "/".join(parts[:3])
    else:
        # If the URI doesn't follow the expected pattern, just return it as is.
        return uri

def load_conceptnet_lite_csv(file_path):
    """
    Loads the filtered ConceptNet Lite CSV into a NetworkX graph.

    Parameters:
    - file_path (str): Path to the filtered CSV file.

    Returns:
    - G (networkx.Graph): The constructed undirected graph with canonicalized nodes.
    """
    print("Loading ConceptNet Lite into NetworkX...")
    G = nx.Graph()

    try:
        # Read the CSV using pandas
        df = pd.read_csv(file_path, sep=',', encoding='utf-8')
    except Exception as e:
        print(f"Error reading filtered CSV: {e}")
        return G

    # Iterate through the DataFrame and add edges
    print("Adding edges to NetworkX...")
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Adding edges"):
        start = canonicalize_node(row['start'])
        end = canonicalize_node(row['end'])
        rel = row['rel']
        G.add_edge(start, end, relation=rel)

    print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G


# Loading ConceptNet with NetworkX

In [ ]:
# Define the path to the filtered CSV
FILTERED_CSV_FILE = "/content/drive/MyDrive/conceptnet-lite[RelatedTo,FormOf,Synonym].csv"

# Load the graph
G = load_conceptnet_lite_csv(FILTERED_CSV_FILE)

Loading ConceptNet Lite into NetworkX...
Adding edges to NetworkX...


Adding edges: 100%|██████████| 2304597/2304597 [03:28<00:00, 11067.08it/s]


Graph has 948526 nodes and 2052758 edges.


# Output Labels (hazard/prod categories)

In [ ]:
json_file_path = 'SemEval-Task9/extracted_keywords_hazard_category.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

df_1 = pd.DataFrame(list(data.items()), columns=['hazard_category', 'extracted_terms'])
hazards = dict(zip(df_1['hazard_category'], df_1['extracted_terms']))

print(hazards)

json_file_path = 'SemEval-Task9/extracted_keywords_product_category.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

df_2 = pd.DataFrame(list(data.items()), columns=['product_category', 'extracted_terms'])
products = dict(zip(df_2['product_category'], df_2['extracted_terms']))
print(products)

{'allergens': ['allergens'], 'chemical': ['chemical'], 'food additives and flavourings': ['food additives', 'flavourings'], 'foreign bodies': ['foreign bodies', 'bodies'], 'fraud': ['fraud'], 'migration': ['migration'], 'organoleptic aspects': ['organoleptic', 'aspects'], 'other hazard': ['hazard'], 'packaging defect': ['defect']}
{'alcoholic beverages': ['alcoholic beverage', 'beverage', 'alcoholic', 'beverages'], 'cereals and bakery products': ['cereals', 'bakery', 'products'], 'cocoa and cocoa preparations, coffee and tea': ['cocoa', 'coffee', 'tea'], 'dietetic foods, food supplements, fortified foods': ['dietetic', 'fortified', 'food', 'supplements'], 'fats and oils': ['fats', 'oils'], 'feed materials': ['materials'], 'food additives and flavourings': ['additive', 'flavourings', 'food additives'], 'food contact materials': ['food', 'contact', 'materials'], 'fruits and vegetables': ['fruits', 'vegetables'], 'herbs and spices': ['herbs', 'spices'], 'honey and royal jelly': ['honey', 

# Input Labels (title keywords)

In [ ]:
json_file_path = 'SemEval-Task9/extracted_keywords.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(list(data.items()), columns=['title', 'extracted_terms'])

# Experimentation

## Removing ban words

In [ ]:
def clean_categories(categories, remove_strings):
    """
    Remove any keywords containing specified strings from both category keys and their associated keywords.

    Parameters:
    - categories (dict): Dictionary mapping category labels to their associated keywords.
    - remove_strings (list): List of strings to be removed from both category keys and their associated keywords.

    Returns:
    - cleaned_categories (dict): Modified dictionary with specified strings removed from each category.
    """
    cleaned_categories = {}

    for label, keywords in categories.items():

        # Filter out keywords containing any of the specified strings
        filtered_keywords = [kw for kw in keywords if all(remove_string != kw for remove_string in remove_strings)]

        # Only add to the dictionary if there are any keywords left after filtering
        if filtered_keywords:
            cleaned_categories[label] = filtered_keywords

    return cleaned_categories


In [ ]:
def remove_terms_from_extracted_terms(df, terms_to_remove):
    """Removes specified terms from the 'extracted_terms' column of a DataFrame.

    Args:
        df: The input DataFrame.
        terms_to_remove: A list of terms to remove.

    Returns:
        A new DataFrame with the specified terms removed from the 'extracted_terms' column.
    """

    # Create a copy of the input DataFrame to avoid modifying the original DataFrame.
    df_cleaned = df.copy()

    # Remove the terms from the 'extracted_terms' column using a list comprehension.
    df_cleaned['extracted_terms'] = df_cleaned['extracted_terms'].apply(
        lambda terms: [term for term in terms if term not in terms_to_remove]
    )

    return df_cleaned

In [ ]:
to_remove = ['good', 'white', 'recall', 'selected', 'stores', 'sold', 'food', 'materials', 'beverage', 'beverages', 'products', 'mixed', 'issues', 'mix', 'alert', 'health', 'product']
print(hazards)
hazards = clean_categories(hazards, to_remove)


print(products)
products = clean_categories(products, to_remove)

print(df.head())

df_cleaned = remove_terms_from_extracted_terms(df, to_remove)

df = df_cleaned
df.head()

{'allergens': ['allergens'], 'chemical': ['chemical'], 'food additives and flavourings': ['food additives', 'flavourings'], 'foreign bodies': ['foreign bodies', 'bodies'], 'fraud': ['fraud'], 'migration': ['migration'], 'organoleptic aspects': ['organoleptic', 'aspects'], 'other hazard': ['hazard'], 'packaging defect': ['defect']}
{'alcoholic beverages': ['alcoholic beverage', 'alcoholic'], 'cereals and bakery products': ['cereals', 'bakery'], 'cocoa and cocoa preparations, coffee and tea': ['cocoa', 'coffee', 'tea'], 'dietetic foods, food supplements, fortified foods': ['dietetic', 'fortified', 'supplements'], 'fats and oils': ['fats', 'oils'], 'food additives and flavourings': ['additive', 'flavourings', 'food additives'], 'food contact materials': ['contact'], 'fruits and vegetables': ['fruits', 'vegetables'], 'herbs and spices': ['herbs', 'spices'], 'honey and royal jelly': ['honey', 'royal jelly'], 'ices and desserts': ['desserts', 'ices'], 'meat, egg and dairy products': ['meat',

,title,extracted_terms
0,2006 - Consumer Alert: Listeria Contamination ...,"[listeria, milk, contamination, raw milk]"
1,2006 - Pinnacle Foods Corporation Issues Aller...,"[corporation, allergy, pinnacle, foods, muffin..."
2,2006 - Pinnacle Foods Group Inc. Issues Allerg...,"[egg, allergy, frozen, undeclared]"
3,2006 - pinnacle foods group inc. issues allerg...,"[allergy, brownie]"
4,2007 - Agriculture Department Warns Consumers ...,"[butler, dairy, county, tainted, raw, milk, wa..."


# BFS

In [ ]:
from collections import deque

def multi_source_bfs(G, start_nodes, max_distance=3):
    """
    Perform BFS from multiple start nodes up to max_distance.

    Parameters:
    - G (networkx.Graph): The graph to traverse.
    - start_nodes (list): List of node identifiers to start BFS from.
    - max_distance (int): Maximum distance to traverse.

    Returns:
    - distance (dict): Dictionary mapping nodes to their shortest distance from any start node.
    """
    distance = {}
    queue = deque()

    # Initialize the queue and distance dictionary with start nodes
    for node in start_nodes:
        if node in G:
            distance[node] = 0
            queue.append((node, 0))
        else:
            print(f"Warning: Start node {node} not in graph.")

    while queue:
        current_node, current_distance = queue.popleft()

        if current_distance >= max_distance:
            continue

        for neighbor in G.neighbors(current_node):
            if neighbor not in distance:
                distance[neighbor] = current_distance + 1
                queue.append((neighbor, current_distance + 1))

    return distance

# Testing

In [ ]:
# Define start concepts
start_concepts = ["alcohol"]  # Modify this list as needed

# Convert start concepts to ConceptNet URIs
start_uris = [f"/c/en/{concept.lower()}" for concept in start_concepts]

print(start_uris)

['/c/en/alcohol']


In [ ]:
# Define the maximum distance for BFS
max_distance = 5  # Adjust as needed

# Perform BFS
dist_map = multi_source_bfs(G, start_uris, max_distance)

In [ ]:
print(len(dist_map))

825502


# Verify if node in ConceptNet

In [ ]:
def verify_nodes(G, nodes):
    """
    Verifies if the given nodes exist in the graph.

    Parameters:
    - G (networkx.Graph or networkx.DiGraph): The graph.
    - nodes (list): List of ConceptNet URIs to verify.

    Returns:
    - existing_nodes (list): Nodes that exist in the graph.
    - missing_nodes (list): Nodes that do not exist in the graph.
    """
    existing_nodes = []
    missing_nodes = []
    for node in nodes:
        if node in G:
            existing_nodes.append(node)
        else:
            missing_nodes.append(node)
    return existing_nodes, missing_nodes

In [ ]:
# Define start concepts
start_concepts = ["dietetical"]  # Modify this list as needed

# Convert start concepts to ConceptNet URIs
start_uris = [f"/c/en/{concept.lower()}" for concept in start_concepts]

print(start_uris)

# Verify if start URIs are in the graph
existing, missing = verify_nodes(G, start_uris)

print("\n=== Node Verification ===")
for node in existing:
    print(f"{node} exists in the graph.")
for node in missing:
    print(f"{node} does NOT exist in the graph.")

['/c/en/dietetical']

=== Node Verification ===
/c/en/dietetical exists in the graph.


# Subgraph generation

In [ ]:
print(products)

# Convert product and hazard keywords to ConceptNet URIs
product_uris = {cat: [f"/c/en/{kw.replace(' ', '_').lower()}" for kw in kws] for cat, kws in products.items()}
hazard_uris = {cat: [f"/c/en/{kw.replace(' ', '_').lower()}" for kw in kws] for cat, kws in hazards.items()}

# Generate subgraphs for products and hazards
product_subgraphs = {cat: multi_source_bfs(G, uris, max_distance) for cat, uris in product_uris.items()}
hazard_subgraphs = {cat: multi_source_bfs(G, uris, max_distance) for cat, uris in hazard_uris.items()}

{'alcoholic beverages': ['alcoholic beverage', 'alcoholic'], 'cereals and bakery products': ['cereals', 'bakery'], 'cocoa and cocoa preparations, coffee and tea': ['cocoa', 'coffee', 'tea'], 'dietetic foods, food supplements, fortified foods': ['dietetic', 'fortified', 'supplements'], 'fats and oils': ['fats', 'oils'], 'food additives and flavourings': ['additive', 'flavourings', 'food additives'], 'food contact materials': ['contact'], 'fruits and vegetables': ['fruits', 'vegetables'], 'herbs and spices': ['herbs', 'spices'], 'honey and royal jelly': ['honey', 'royal jelly'], 'ices and desserts': ['desserts', 'ices'], 'meat, egg and dairy products': ['meat', 'egg', 'dairy products'], 'nuts, nut products and seeds': ['nut', 'nuts', 'seeds'], 'pet feed': ['pet'], 'prepared dishes and snacks': ['dishes', 'snacks', 'prepared'], 'seafood': ['seafood'], 'soups, broths, sauces and condiments': ['broth', 'broths', 'soups', 'soup', 'sauces', 'sauce', 'condiments', 'condiment'], 'sugars and syr

# Classification

In [ ]:
def classify_title_keywords(df, categories, max_distance=5):
    """
    Classify titles into categories based on the minimum distance between title keywords and pre-generated category subgraphs.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'title' and 'extracted_terms' columns.
    - categories (dict): Dictionary mapping category labels to their distance dictionaries (pre-generated subgraphs).
    - max_distance (int): Maximum BFS distance to consider.

    Returns:
    - classifications (dict): Dictionary mapping titles to their classified category.
    """
    classifications = {}

    # Iterate over each title in the DataFrame and classify
    for _, row in df.iterrows():
        title = row['title']
        keywords = row['extracted_terms']
        #print(keywords)
        #print(f"Processing title: {title}, Keywords: {keywords}")

        min_distance = float('inf')
        assigned_category = None

        # Compute the minimum distance for each category to the title's keywords
        for category, distances in categories.items():

            # MIN CODE
            # category_min_distance = float('inf')

            # Get the minimum distance to any keyword in the title
            # for keyword in keywords:
            #     keyword_uri = f"/c/en/{keyword.replace(' ', '_').lower()}"
            #     distance = distances.get(keyword_uri, float('inf'))
            #     if distance < category_min_distance:
            #         category_min_distance = distance
            #         #print(f"Category: {category}, Keyword: {keyword}, Distance: {distance}")

            # AVG CODE

            # Calculate the average distance to all keywords in the title for each category
            category_average_distance = float('inf')

            if keywords:
                distances_sum = 0
                valid_distances = 0

                for keyword in keywords:
                    #print(keyword)
                    keyword_uri = f"/c/en/{keyword.replace(' ', '_').lower()}"
                    distance = distances.get(keyword_uri, float('inf'))

                    # Only consider valid distances
                    if distance != float('inf'):
                        #print(f"Category: {category}, Keyword: {keyword}, Distance: {distance}")
                        distances_sum += distance
                        valid_distances += 1


                # Compute average distance if there are valid distances
                if valid_distances > 0:
                    category_average_distance = distances_sum
                    #print(f"Category: {category}, Average Distance: {category_average_distance}")


            # Check if this category has a smaller average distance than previously found categories
            if category_average_distance < min_distance:
                min_distance = category_average_distance
                assigned_category = category

            # # Check if this category is closer than previously found categories
            # if category_min_distance < min_distance:
            #     min_distance = category_min_distance
            #     assigned_category = category

        # Assign the category with the overall shortest distance
        classifications[title] = assigned_category

    return classifications


In [ ]:
categories = product_subgraphs

classifications = classify_title_keywords(df, categories, G)

classifications

{'2006 - Consumer Alert: Listeria Contamination in Raw Milk': 'meat, egg and dairy products',
 '2006 - Pinnacle Foods Corporation Issues Allergy Alert on Bakery-Style Chocolate Chip Muffin Mix': 'cocoa and cocoa preparations, coffee and tea',
 '2006 - Pinnacle Foods Group Inc. Issues Allergy Alert on Undeclared Egg in Frozen': 'meat, egg and dairy products',
 '2006 - pinnacle foods group inc. issues allergy alert on brownie mix': 'nuts, nut products and seeds',
 '2007 - Agriculture Department Warns Consumers of Tainted Raw Milk Sold by Butler County Dairy': 'meat, egg and dairy products',
 '2007 - WholeSoy & Co. Expands National Allergy Alert on Undeclared Dairy in Blueberry and Mixed Berry Yogurt': 'meat, egg and dairy products',
 '2007 - Winn-Dixie Stores, Inc. Issues Allergy Alert on Mislabeled Prestige Chocolate Ice Cream': 'cocoa and cocoa preparations, coffee and tea',
 '2007 - private selectionÂ® ice cream recalled from ralphs and food 4 less stores in california': 'meat, egg an

In [ ]:
df[:10]

,title,extracted_terms
0,2006 - Consumer Alert: Listeria Contamination ...,"[listeria, milk, contamination, raw milk]"
1,2006 - Pinnacle Foods Corporation Issues Aller...,"[corporation, allergy, pinnacle, foods, muffin..."
2,2006 - Pinnacle Foods Group Inc. Issues Allerg...,"[egg, allergy, frozen, undeclared]"
3,2006 - pinnacle foods group inc. issues allerg...,"[allergy, brownie]"
4,2007 - Agriculture Department Warns Consumers ...,"[butler, dairy, county, tainted, raw, milk, wa..."
5,2007 - WholeSoy & Co. Expands National Allergy...,"[allergy, dairy, blueberry, yogurt]"
6,"2007 - Winn-Dixie Stores, Inc. Issues Allergy ...","[allergy, issue, chocolate]"
7,2007 - private selectionÂ® ice cream recalled ...,"[store, recalled, cream, ice, stores, private,..."
8,2008 - Barry Callebaut USA LLC Issues Allergen...,"[chocolate, chip, milk, protein, allergen, iss..."
9,2008 - dreyer's issues voluntary recall of slo...,"[undeclared, pecans, recall, dreyer, caramel, ..."


In [ ]:
# def classify_title_keywords(df, products, hazards, G, max_distance=5):
#     """
#     Classify title keywords into categories based on distance to product and hazard subgraphs.

#     Parameters:
#     - df (pd.DataFrame): DataFrame containing 'title' and 'extracted_terms' columns.
#     - products (dict): Dictionary mapping product categories to their keywords.
#     - hazards (dict): Dictionary mapping hazard categories to their keywords.
#     - G (networkx.Graph): The ConceptNet graph.
#     - max_distance (int): Maximum BFS distance to consider.

#     Returns:
#     - classifications (dict): Dictionary mapping titles to their classified category.
#     """
#     classifications = {}


#     dc = 0

#     # Iterate over each title and classify
#     for _, row in df.iterrows():
#         title = row['title']
#         keywords = row['extracted_terms']
#         min_distance = float('inf')
#         assigned_category = None

#         for keyword in keywords:
#             keyword_uri = f"/c/en/{keyword.replace(' ', '_').lower()}"

#             # Check distance in product categories
#             for cat, distances in product_distances.items():
#                 distance = distances.get(keyword_uri, float('inf'))
#                 if distance < min_distance:
#                     min_distance = distance
#                     assigned_category = f"Product: {cat}"


#             # Check distance in hazard categories
#             for cat, distances in hazard_distances.items():
#                 distance = distances.get(keyword_uri, float('inf'))
#                 if distance == 'inf':
#                   dc+=1
#                 if distance < min_distance:
#                     min_distance = distance
#                     assigned_category = f"Hazard: {cat}"

#         # Assign the category with the shortest distance
#         classifications[title] = assigned_category

#     return classifications

# Working

In [ ]:
classifications = classify_title_keywords(df, products, hazards, G)

In [ ]:
classifications

{'2006 - Consumer Alert: Listeria Contamination in Raw Milk': 'Product: alcoholic beverages',
 '2006 - Pinnacle Foods Corporation Issues Allergy Alert on Bakery-Style Chocolate Chip Muffin Mix': 'Product: dietetic foods, food supplements, fortified foods',
 '2006 - Pinnacle Foods Group Inc. Issues Allergy Alert on Undeclared Egg in Frozen': 'Product: meat, egg and dairy products',
 '2006 - pinnacle foods group inc. issues allergy alert on brownie mix': 'Product: other food product / mixed',
 '2007 - Agriculture Department Warns Consumers of Tainted Raw Milk Sold by Butler County Dairy': 'Product: alcoholic beverages',
 '2007 - WholeSoy & Co. Expands National Allergy Alert on Undeclared Dairy in Blueberry and Mixed Berry Yogurt': 'Product: dietetic foods, food supplements, fortified foods',
 '2007 - Winn-Dixie Stores, Inc. Issues Allergy Alert on Mislabeled Prestige Chocolate Ice Cream': 'Product: cocoa and cocoa preparations, coffee and tea',
 '2007 - private selectionÂ® ice cream reca

# Random Sh

In [ ]:
import gzip

# Define the target string
target = "/c/en/oils/"

file_name = "/content/conceptnet-lite.csv"

# Decompress and process line by line
output_file = "filtered_results.csv"
with gzip.open(file_name, "rt", encoding="utf-8") as f_in, open(output_file, "w", encoding="utf-8") as f_out:
    for line in f_in:
        if target in line:
            f_out.write(line)  # Write matching lines to a new CSV file

print(f"Filtered lines containing '{target}' are saved to {output_file}")

BadGzipFile: Not a gzipped file (b'@i')

In [ ]:
file_name = "/content/conceptnet-lite.csv"

!grep '/c/en/oils/' /content/conceptnet-lite.csv

"/a/[/r/FormOf/,/c/en/oils/,/c/en/oil/n/]",/r/FormOf,/c/en/oils,/c/en/oil/n,"{""dataset"": ""/d/wiktionary/de"", ""license"": ""cc:by-sa/4.0"", ""sources"": [{""contributor"": ""/s/resource/wiktionary/de"", ""process"": ""/s/process/wikiparsec/2""}], ""weight"": 1.0}"
"/a/[/r/FormOf/,/c/en/oils/n/,/c/en/oil/]",/r/FormOf,/c/en/oils/n,/c/en/oil,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc:by-sa/4.0"", ""sources"": [{""contributor"": ""/s/resource/wiktionary/en"", ""process"": ""/s/process/wikiparsec/2""}], ""weight"": 1.0}"
"/a/[/r/FormOf/,/c/en/oils/v/,/c/en/oil/]",/r/FormOf,/c/en/oils/v,/c/en/oil,"{""dataset"": ""/d/wiktionary/en"", ""license"": ""cc:by-sa/4.0"", ""sources"": [{""contributor"": ""/s/resource/wiktionary/en"", ""process"": ""/s/process/wikiparsec/2""}], ""weight"": 1.0}"
